<a href="https://colab.research.google.com/github/HOKAGE149/image_public_baseline/blob/main/%E7%94%BB%E5%83%8F_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## まとまってる記事

https://qiita.com/ground0state/items/c1d705ca2ee329cdfae4

https://qiita.com/fam_taro/items/df8656a6c3b277f58781

## インストール

In [ ]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
! pip install --quiet "scikit-learn" "datasets" "scipy" "torchmetrics" "transformers" "torch==1.9" "pytorch-lightning==1.2.8"

In [ ]:
!pip install -q timm

In [ ]:
!pip uninstall -y albumentations
!pip install -q albumentations==0.5.2

Found existing installation: albumentations 0.5.2
Uninstalling albumentations-0.5.2:
  Successfully uninstalled albumentations-0.5.2


In [ ]:

# !pip install -U git+https://github.com/albu/albumentations > /dev/null 


In [ ]:
from datetime import datetime
from typing import Optional

import datasets
import torch
from pytorch_lightning import LightningDataModule, LightningModule, seed_everything, Trainer
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers.csv_logs import CSVLogger
from torch.utils.data import DataLoader
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)
# ========================================
# library
# ========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel,AutoConfig
import transformers
from transformers import RobertaModel,RobertaTokenizer
from transformers import AlbertModel,AlbertTokenizer
from transformers import XLNetModel,XLNetTokenizer,XLNetConfig
from transformers import DebertaModel, DebertaTokenizer
from transformers import ElectraModel, ElectraTokenizer, ElectraForSequenceClassification
from transformers import BartModel,BertModel,BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import MPNetModel,MPNetTokenizer
from transformers import FunnelBaseModel,FunnelTokenizer,FunnelModel
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
from transformers import T5EncoderModel,T5Tokenizer
import logging
import sys
from contextlib import contextmanager
import time
import random
from tqdm import tqdm
import os
import pickle
import gc
from sklearn import model_selection
AVAIL_GPUS = min(1, torch.cuda.device_count())

# Hide Warning
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Python Libraries
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np
import os
import random
import glob
import math


# Visualizations
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import plotly.express as px
%matplotlib inline
sns.set(style="whitegrid")

# Utilities and Metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Pytorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.optimizer import Optimizer, required 


# Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

# Pytorch Image Models
import timm

# Image Augmentation Library
import albumentations
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as AF



In [ ]:
# %cd '/content/drive/MyDrive/Colab Notebooks/SETI/src'

In [ ]:
!pip install --upgrade -q wandb

In [ ]:
# Weights and Biases Tool
import wandb
wandb.login()

True

In [ ]:
albumentations.__version__

'0.5.2'

## データロード30分

In [1]:
# !chmod 600 /root/.kaggle/kaggle.json
# !pip uninstall -y kaggle
# !pip install --upgrade pip
# !pip install kaggle==1.5.6

# GCS_DS_PATH = "gs://kds-06ebe33f115002b87e00190a4afa61624bf9aaf3b7f192044f43ffef"
# !gsutil -m cp -r {GCS_DS_PATH}/"train" .

## config

In [ ]:
class dict2(dict): 
    def __init__(self, *args, **kwargs): 
        super().__init__(*args, **kwargs) 
        self.__dict__ = self 
cfg = dict2({'General': 
            dict2({'seed': 1996, 
                  'debug': False, 
                   'compe_name': 'SETI',
                  'project': 'project001', 
                  'epoch': 10, 
                  'gpus': 1, 
                  'multi_gpu_mode': False, 
                  'fp16': False,
                  'amp_level': False,
                   'image_size': 320,
                   'save_dir': '/content',
                   'tb_logger_dir': './lightning_log'
                }), 
          'Data': 
            dict2({'dataset': 
                    dict2({'fold': 0,
                           'nfolds': 5,
                           'test_csv': '/content/drive/MyDrive/Colab Notebooks/SETI/input/sample_submission.csv',
                           'train_csv': '/content/drive/MyDrive/Colab Notebooks/SETI/input/train_labels_paths.csv',
                           'target_col': 'target'}),
                   'dataloader':
                    dict2({'train_batch_size': 32,
                           'valid_batch_size': 32,
                           'test_batch_size': 32,
                           'train_num_workers': 8,
                           'valid_num_workers': 8,
                           'test_num_workers': 8})
                    }),
          'Model': 
            dict2({'name': 'efficientnet_b0',
                   'pretrained': True,
                   'out_features': 1,
                   'input_channels': 1,
                   }),
          'Loss':
            dict2({'name': 'AUC'}),
          'Optimizer':
            dict2({'name': 'AdamW',
                   'lr': 1e-3}),
          'Scheduler':
            dict2({'name': 'CosineAnnealingLR',
                   'T_max': 10,
                   'min_lr': 1e-6}),
          'Augmentation': True
})

params = {
    'seed': 42,
    'model': 'efficientnet_b0',
    'size' : 320,
    'inp_channels': 1,
    'lr': 1e-3,
    'weight_decay': 1e-6,
    'batch_size': 32,
    'num_workers' : 8,
    'epochs': 5,
    'out_features': 1,
    'name': 'CosineAnnealingLR',
    'T_max': 10,
    'min_lr': 1e-6,
    'nfolds': 5,
    'precision': 16
}

In [ ]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(cfg.General.seed)

## Load Train and Test






In [ ]:
import pandas as pd
train_dir = ('/content/train')
# test_dir = ('../input/seti-breakthrough-listen/test')
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SETI/input/train_labels.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SETI/input/sample_submission.csv')

In [ ]:
def return_filpath(name, folder=train_dir):
    path = os.path.join(folder, name[0], f'{name}.npy')
    return path

In [ ]:
train_df['image_path'] = train_df['id'].apply(lambda x: return_filpath(x))
# test_df['image_path'] = test_df['id'].apply(lambda x: return_filpath(x, folder=test_dir))
train_df.head()

,id,target,image_path
0,0000799a2b2c42d,0,/content/train/0/0000799a2b2c42d.npy
1,00042890562ff68,0,/content/train/0/00042890562ff68.npy
2,0005364cdcb8e5b,0,/content/train/0/0005364cdcb8e5b.npy
3,0007a5a46901c56,0,/content/train/0/0007a5a46901c56.npy
4,0009283e145448e,0,/content/train/0/0009283e145448e.npy


In [ ]:
# train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/SETI/input/train_labels_paths.csv')

## GridMask Data Augmentation

In [ ]:
class GridMask(DualTransform):

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = AF.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

## Image Augmentation

In [ ]:
def get_train_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(params['size'],params['size']),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=180, p=0.7),
            albumentations.RandomBrightness(limit=0.6, p=0.5),
            albumentations.Cutout(
                num_holes=10, max_h_size=12, max_w_size=12,
                fill_value=0, always_apply=False, p=0.5
            ),
            albumentations.ShiftScaleRotate(
                shift_limit=0.25, scale_limit=0.1, rotate_limit=0
            ),
            albumentations.OneOf([
                GridMask(num_grid=3, mode=0, rotate=15),
                GridMask(num_grid=3, mode=2, rotate=15),
                                ], p=0.7),
            ToTensorV2(p=1.0),
        ]
    )

def get_valid_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(params['size'],params['size']),
            ToTensorV2(p=1.0)
        ]
    )

def get_test_transforms():
        return albumentations.Compose(
            [
                albumentations.Resize(params['size'],params['size']),
                ToTensorV2(p=1.0)
            ]
        )

## Custom Dataset

In [ ]:
class LitDataset(Dataset):
    def __init__(self, images_filepaths, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = np.load(image_filepath).astype(np.float32)
        image = np.vstack(image).transpose((1, 0))
            
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
        
        label = torch.tensor(self.targets[idx]).float()
        return image, label

<p p style = "font-family: garamond; font-size:30px; font-style: normal;background-color: #f6f5f5; color :#6666ff; border-radius: 10px 10px; text-align:center">Custom Callback for Viewing Predictions in W&B <br> Code Credit <a href = 'https://www.kaggle.com/ayuraj/use-pytorch-lightning-with-weights-and-biases'>Ayush Thakur </a></p>


In [ ]:
class ImagePredictionLogger(Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples
        
    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        trainer.logger.experiment[1].log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Target:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
            }, commit=False)

## Ranger Optimizer with Gradient Centralization


In [ ]:
# https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer

class Ranger(Optimizer):

    def __init__(self, params, lr=1e-3,                       # lr
                 alpha=0.5, k=6, N_sma_threshhold=5,           # Ranger options
                 betas=(.95, 0.999), eps=1e-5, weight_decay=0,  # Adam options
                 # Gradient centralization on or off, applied to conv layers only or conv + fc layers
                 use_gc=True, gc_conv_only=False
                 ):

        # parameter checks
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        if not lr > 0:
            raise ValueError(f'Invalid Learning Rate: {lr}')
        if not eps > 0:
            raise ValueError(f'Invalid eps: {eps}')


        # prep defaults and init torch.optim base
        defaults = dict(lr=lr, alpha=alpha, k=k, step_counter=0, betas=betas,
                        N_sma_threshhold=N_sma_threshhold, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)

        # adjustable threshold
        self.N_sma_threshhold = N_sma_threshhold

        # look ahead params

        self.alpha = alpha
        self.k = k

        # radam buffer for state
        self.radam_buffer = [[None, None, None] for ind in range(10)]

        # gc on or off
        self.use_gc = use_gc

        # level of gradient centralization
        self.gc_gradient_threshold = 3 if gc_conv_only else 1

        print(
            f"Ranger optimizer loaded. \nGradient Centralization usage = {self.use_gc}")
        if (self.use_gc and self.gc_gradient_threshold == 1):
            print(f"GC applied to both conv and fc layers")
        elif (self.use_gc and self.gc_gradient_threshold == 3):
            print(f"GC applied to conv layers only")

    def __setstate__(self, state):
        print("set state called")
        super(Ranger, self).__setstate__(state)

    def step(self, closure=None):
        loss = None

        # Evaluate averages and grad, update param tensors
        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()

                if grad.is_sparse:
                    raise RuntimeError(
                        'Ranger optimizer does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]  # get state dict for this param

                if len(state) == 0:  

                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)

                    # look ahead weight storage now in state dict
                    state['slow_buffer'] = torch.empty_like(p.data)
                    state['slow_buffer'].copy_(p.data)

                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(
                        p_data_fp32)

                # begin computations
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                # GC operation for Conv layers and FC layers
                if grad.dim() > self.gc_gradient_threshold:
                    grad.add_(-grad.mean(dim=tuple(range(1, grad.dim())), keepdim=True))

                state['step'] += 1

                # compute variance mov avg
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                # compute mean moving avg
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                buffered = self.radam_buffer[int(state['step'] % 10)]

                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * \
                        state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma
                    if N_sma > self.N_sma_threshhold:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (
                            N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay']
                                     * group['lr'], p_data_fp32)

                # apply lr
                if N_sma > self.N_sma_threshhold:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size *
                                         group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

                # integrated look ahead...
                # we do it at the param level instead of group level
                if state['step'] % group['k'] == 0:
                    # get access to slow param tensor
                    slow_p = state['slow_buffer']
                    # (fast weights - slow weights) * alpha
                    slow_p.add_(self.alpha, p.data - slow_p)
                    # copy interpolated weights to RAdam param tensor
                    p.data.copy_(slow_p)

        return loss

## Pytorch Lightning DataModule


In [ ]:
class DataModule(pl.LightningDataModule):

    def __init__(self, cfg):
        super().__init__()


        # Config ファイルからの読み込み
        self.cfg_dataset = cfg.Data.dataset
        self.cfg_augmentation = cfg.Augmentation
        self.cfg_dataloader = cfg.Data.dataloader

        self.test_df = None
        self.train_df = None
        self.valid_df = None   


    def get_test_df(self):
        return pd.read_csv(self.cfg_dataset.test_csv, index_col=0).reset_index(drop = True)

    def split_train_valid_df(self):
        df = pd.read_csv(self.cfg_dataset.train_csv, index_col=0).reset_index(drop = True)

        # Split
        skf = StratifiedKFold(n_splits=self.cfg_dataset.nfolds, shuffle=True, random_state=cfg.General.seed)
        for n, (train_index, val_index) in enumerate(skf.split(df, df[self.cfg_dataset.target_col])):
            df.loc[val_index, "fold"] = int(n)
        df["fold"] = df["fold"].astype(int)

        fold = int(self.cfg_dataset.fold)
        train_df = df[df["fold"] != fold].reset_index(drop=True)
        valid_df = df[df["fold"] == fold].reset_index(drop=True)
        return train_df, valid_df

    # 必ず呼び出される関数
    def setup(self, stage=False):
        self.test_df = self.get_test_df()
        train_df, valid_df = self.split_train_valid_df()
        self.train_df = train_df
        self.valid_df = valid_df
        

        self.train_dataset = LitDataset(
        images_filepaths=self.train_df['image_path'].values,
        targets=self.train_df[self.cfg_dataset.target_col].values,
        transform=get_train_transforms()
            )
        
        self.val_dataset = LitDataset(
        images_filepaths=self.valid_df['image_path'].values,
        targets=self.valid_df[self.cfg_dataset.target_col].values,
        transform=get_valid_transforms()
            )

        # self.test_dataset = LitDataset(
        # images_filepaths = self.test_df['image_path'].values,
        # targets = self.test_df[self.cfg_dataset.target_col].values,
        # transform = get_test_transforms()
        #     )

    def train_dataloader(self):
        return DataLoader(
        self.train_dataset,
        batch_size=self.cfg_dataloader.train_batch_size,
        shuffle=True,
        num_workers=self.cfg_dataloader.train_num_workers,
        pin_memory=True
            )

    def val_dataloader(self):
        return DataLoader(
        self.val_dataset,
        batch_size=self.cfg_dataloader.valid_batch_size,
        shuffle=False,
        num_workers=self.cfg_dataloader.valid_num_workers,
        pin_memory=True
            )

    def test_dataloader(self):
        return DataLoader(
        self.test_dataset, batch_size=self.cfg_dataloader.test_batch_size,
        shuffle=False, num_workers=self.cfg_dataloader.test_num_workers,
        pin_memory=True
            )

In [ ]:
# print(train_df)
# LitDataset(
#         images_filepaths = train_df['image_path'].values,
#         targets=train_df[cfg.Data.dataset.target_col].values,
#         # transform=get_train_transforms()
#             )[0]

## Pytorch Lightning Module

In [ ]:
# !kaggle competitions download -c seti-breakthrough-listen

In [ ]:
# !kaggle datasets download -d snaker/train-7

In [ ]:
# print(labels_data)
# print(output_data.sigmoid())

In [ ]:
# labels_data.shape

In [ ]:
# model = MyLightningModule(cfg)
# data_next = next(iter(data_module.train_dataloader()))
# x_data  = data_next[0]
# y_data = data_next[1]
# output_data = model(x_data)
# labels_data = y_data.unsqueeze(1)
# # print(labels_data)
# # print(output_data)
# print(nn.BCEWithLogitsLoss()(output_data, labels_data))
# print(roc_auc_score(labels_data.detach().cpu(), output_data.sigmoid().detach().cpu()) )

In [ ]:
class MyLightningModule(pl.LightningModule):

    def __init__(self, cfg):
        super().__init__()
        self.save_hyperparameters()
        
        self.model = timm.create_model(cfg.Model.name, pretrained=cfg.Model.pretrained,
                                       in_chans=cfg.Model.input_channels)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, cfg.Model.out_features, bias=True) 
        
        self.criterion = nn.BCEWithLogitsLoss()
        self.cfg_optim = cfg.Optimizer
        self.cfg_schedule = cfg.Scheduler
                
    def forward(self, x):
        output = self.model(x)
        return output


    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        labels = y.unsqueeze(1)
        loss = self.criterion(output, labels)
        
        try:
            auc=roc_auc_score(labels.detach().cpu(), output.sigmoid().detach().cpu()) 

            self.log("auc", auc, on_step= True, prog_bar=True, logger=True)
            self.log("Train Loss", loss, on_step= True,prog_bar=True, logger=True)
        except:
            pass
        return {"loss": loss, "predictions": output, "labels": labels}

    def training_epoch_end(self, outputs):

        preds = []
        labels = []
        
        for output in outputs:
            
            preds += output['predictions']
            labels += output['labels']

        labels = torch.stack(labels)
        preds = torch.stack(preds)
        # try:
        #     train_auc=roc_auc_score(labels.detach().cpu(), preds.sigmoid().detach().cpu())
        #     self.log("mean_train_auc", train_auc, prog_bar=True, logger=True)
        # except:
        #     pass
        train_auc=roc_auc_score(labels.detach().cpu(), preds.sigmoid().detach().cpu())
        self.log("mean_train_auc", train_auc, prog_bar=True, logger=True)        

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        labels = y.unsqueeze(1)
        loss = self.criterion(output, labels)
        self.log('val_loss', loss, on_step= True, prog_bar=True, logger=True)
        return {"predictions": output, "labels": labels}
      

    def validation_epoch_end(self, outputs):

        preds = []
        labels = []
        
        for output in outputs:
            preds += output['predictions']
            labels += output['labels']

        labels = torch.stack(labels)
        preds = torch.stack(preds)
        # try:
        #     val_auc=roc_auc_score(labels.detach().cpu(), preds.sigmoid().detach().cpu())
        #     self.log("val_auc", val_auc, prog_bar=True, logger=True)
        # except:
        #     pass    
        val_auc=roc_auc_score(labels.detach().cpu(), preds.sigmoid().detach().cpu())
        self.log("val_auc", val_auc, prog_bar=True, logger=True)    

        

    def test_step(self, batch, batch_idx):
        x = batch        
        output = self(x).sigmoid()
        return output   

    def configure_optimizers(self):

        optimizer = Ranger(self.parameters(), lr = self.cfg_optim.lr)

        scheduler = CosineAnnealingLR(optimizer,
                              T_max=self.cfg_schedule.T_max,
                              eta_min=self.cfg_schedule.min_lr,
                              last_epoch=-1)

        return dict(
          optimizer=optimizer,
          lr_scheduler=scheduler
        )

## Pytorch Lightning Trainer



In [ ]:
# !pip uninstall pytorch-lightning wandb

In [ ]:
# !pip install -qqq pytorch-lightning wandb

In [ ]:
# ! pip install --quiet "scikit-learn" "datasets" "scipy" "torchmetrics" "transformers" "torch==1.9" "pytorch-lightning>=1.3"

In [ ]:
# from pytorch_lightning.plugins.training_type import DDPPlugin

In [ ]:
fold = cfg.Data.dataset.fold
cfg.General.debug = True
print(f"{'='*38} Fold: {fold} {'='*38}")

logger = CSVLogger(save_dir=cfg.General.save_dir, name=cfg.General.compe_name +'_' +cfg.General.project + f"fold_{fold}")
wandb_logger = WandbLogger(name=cfg.General.compe_name +'_' +cfg.General.project + f"_{fold}", project=cfg.General.project, offline = False)
tb_logger = pl_loggers.TensorBoardLogger(cfg.General.tb_logger_dir)

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename=cfg.General.compe_name +'_' +cfg.General.project + "_{epoch:02d}-{fold}-{val_auc:.3f}",
    save_top_k=3,
    verbose=True,
    monitor="val_auc",
    mode="max"
)


model = MyLightningModule(cfg)
data_module = DataModule(cfg)
data_module.setup()
val_samples = next(iter(data_module.val_dataloader()))

early_stopping_callback = EarlyStopping(monitor='val_auc',mode="max", patience=2)




# 
trainer = pl.Trainer(
    gpus= cfg.General.gpus,
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback,
                    ImagePredictionLogger(val_samples)],
    max_epochs=cfg.General.epoch,
    precision=16 if cfg.General.fp16 else 32,
    progress_bar_refresh_rate=1,
    limit_train_batches = 0.005 if cfg.General.debug else 1.0,
    limit_val_batches = 0.005 if cfg.General.debug else 1.0,
    logger=[logger, wandb_logger, tb_logger],
    # logger=wandb_logger,
    # For fast https://pytorch-lightning.readthedocs.io/en/1.3.3/benchmarking/performance.html#
    # plugins=DDPPlugin(find_unused_parameters=False),
)

trainer.fit(model, data_module)

[このコード](https://www.kaggle.com/ryonasuda/pytorch-lightning-grid-mask-ranger-opt-w-b/edit)

In [ ]:
gc.collect()

In [ ]:
pl.__version__

In [ ]:
cfg.General.gpus

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_log/default